In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime

def greet():
    print("Hello, Airflow!")

default_args = {
    "start_date": datetime(2024, 1, 1),
    "catchup": False
}

with DAG("hello_airflow", default_args=default_args, schedule_interval="@daily") as dag:
    task1 = PythonOperator(
        task_id="say_hello",
        python_callable=greet
    )

task1


In [ ]:
# Simple ETL DAG
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.providers.postgres.operators.postgres import PostgresOperator
from datetime import datetime, timedelta
import requests
import pandas as pd
import psycopg2

# Default args
default_args = {
    "owner": "airflow",
    "start_date": datetime(2024, 1, 1),
    "retries": 1,
    "retry_delay": timedelta(minutes=5),
    "catchup": False
}

# Function to extract data
def extract():
    url = "https://jsonplaceholder.typicode.com/posts"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)
    df.to_csv("/tmp/posts.csv", index=False)

# Function to transform data
def transform():
    df = pd.read_csv("/tmp/posts.csv")
    df["title"] = df["title"].str.upper()
    df.to_csv("/tmp/posts_transformed.csv", index=False)

# Function to load data into PostgreSQL
def load():
    conn = psycopg2.connect(
        dbname="your_database",
        user="your_user",
        password="your_password",
        host="localhost",
        port="5432"
    )
    cursor = conn.cursor()
    df = pd.read_csv("/tmp/posts_transformed.csv")
    
    for _, row in df.iterrows():
        cursor.execute(
            "INSERT INTO posts (id, user_id, title, body) VALUES (%s, %s, %s, %s)",
            (row["id"], row["userId"], row["title"], row["body"])
        )
    
    conn.commit()
    cursor.close()
    conn.close()

# Define DAG
with DAG("etl_pipeline", default_args=default_args, schedule_interval="@daily") as dag:
    
    create_table = PostgresOperator(
        task_id="create_table",
        postgres_conn_id="postgres_local",
        sql="""
        CREATE TABLE IF NOT EXISTS posts (
            id INT PRIMARY KEY,
            user_id INT,
            title TEXT,
            body TEXT
        );
        """
    )

    extract_task = PythonOperator(task_id="extract", python_callable=extract)
    transform_task = PythonOperator(task_id="transform", python_callable=transform)
    load_task = PythonOperator(task_id="load", python_callable=load)

    create_table >> extract_task >> transform_task >> load_task
